In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

In [ ]:
np.random.seed(7)

In [ ]:
# whether to use LSTM or MLP
use_LSTM = True

# number of features used in the regression (for MLP)
mlp_num_features = 10
#

# lstm_num_timesteps
lstm_num_timesteps = 10
# lstm_num_features
lstm_num_features = 1
# stateful?
lstm_stateful = False
# use two lstm layers?
lstm_stack_layers = False


batch_size = 1
num_epochs = 1000
num_neurons = 4

# scale the dataset to values between scale_min and scale_max
scale = True
scale_min = -1
scale_max = 1

In [ ]:
filename = '108_7_3.train.csv'
df_train = pd.read_csv(filename, usecols=[1])
ts_train = df_train.values.astype('float64')
df_test = pd.read_csv(filename, usecols=[1])
ts_test = df_test.values.astype('float64')
ts_all = np.append(ts_train, ts_test).reshape(-1,1)
len_overall = len(ts_all)

In [ ]:
len_overall

In [ ]:
ts_train.shape, ts_test.shape

In [ ]:
ts_train[:10]

In [ ]:
ts_test[:10]

In [ ]:
if scale:
    scaler = MinMaxScaler(feature_range=(scale_min, scale_max))
    ts_train = scaler.fit_transform(ts_train)
    ts_test = scaler.transform(ts_test)
    scaler.data_min_, scaler.data_max_

In [ ]:
ts_train[:10]

In [ ]:
ts_test[:10]

In [ ]:
# split into train and test sets
#train_size = int(len(ts) * 0.67)
#test_size = len(ts) - train_size
#ts_train, ts_test = ts[0:train_size,:], ts[train_size:len(ts),:]
#print(len(ts_train), len(ts_test))


In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - window_size):
        a = dataset[i:(i + window_size), 0]
        dataX.append(a)
        dataY.append(dataset[i + window_size, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
def create_windowed(ts, lag):
    df = pd.DataFrame(ts)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df = df.drop(0)
    return df

In [ ]:
if use_LSTM:
    X_train, y_train = create_dataset(ts_train, lstm_num_timesteps)
    X_test, y_test = create_dataset(ts_test, lstm_num_timesteps)
else:
    X_train, y_train = create_dataset(ts_train, mlp_num_features)
    X_test, y_test = create_dataset(ts_test, mlp_num_features)
    
# the train and test matrices end up shorter than the respective timeseries by window_size + 1!
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train[:5,:]

In [ ]:
y_train[:5]

In [ ]:
X_test[:5,:]

In [ ]:
y_test[:5]

In [ ]:
if use_LSTM:
    # reshape input to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
    X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))

In [ ]:
model = Sequential()

if use_LSTM:
    
    # the last state for each sample at index i in a batch will be used as initial state
    # for the sample of index i in the following batch
    if lstm_stateful:
        #
        #
        if lstm_stack_layers:
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True,
                       return_sequences = True))
            model.add(LSTM(num_neurons,
                       stateful = True))
        # 
        else:
            model.add(LSTM(num_neurons,
                       batch_input_shape=(batch_size, X_train.shape[1], X_train.shape[2]),
                       stateful = True))
          
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        
        for i in range(num_epochs):
            print('epoch: ' + str(i))
            # shuffle must be False!
            model.fit(X_train, y_train, nb_epoch = 1, batch_size = batch_size, shuffle = False)
            model.reset_states()
 

    # stateful == False    
    else:        
        
        if lstm_stack_layers:
            # input_dim: dimensionality of the input (alternatively, input_shape)
            # required when using this layer as the first layer in a model
            model.add(LSTM(num_neurons, input_dim = lstm_num_features, return_sequences = True))
            model.add(LSTM(num_neurons))
        # 
        else:
            model.add(LSTM(num_neurons, input_dim = lstm_num_features))
        
        model.add(Dense(1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)
        
   

# feedforward
else:
    
    model.add(Dense(num_neurons, input_dim = mlp_num_features, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)


In [ ]:
test_loss = np.nan
if lstm_stateful:
    test_loss = model.evaluate(X_test, y_test, batch_size = batch_size)
else:
    test_loss = model.evaluate(X_test, y_test, batch_size = X_test.shape[0])
test_loss

In [ ]:
if lstm_stateful:
    pred_train = model.predict(X_train, batch_size = batch_size)
    pred_test = model.predict(X_test, batch_size = batch_size)
else:
    pred_train = model.predict(X_train, batch_size = X_train.shape[0])
    pred_test = model.predict(X_test, batch_size = X_test.shape[0])

In [ ]:
y_train[:10]

In [ ]:
pred_train[:10,0]

In [ ]:
y_test[:10]

In [ ]:
pred_test[:10,0]

In [ ]:
if scale:
    pred_train = scaler.inverse_transform(pred_train)
    y_train = scaler.inverse_transform(y_train.reshape(-1,1))
    pred_test = scaler.inverse_transform(pred_test)
    y_test = scaler.inverse_transform(y_test.reshape(-1,1))


In [ ]:
y_train[:10],pred_train[:10,0]

In [ ]:
y_test[:10],pred_test[:10,0]

In [ ]:
# calculate root mean squared error
rsme_train = math.sqrt(mean_squared_error(y_train, pred_train[:,0]))
print('Train Score: %.2f RMSE' % (rsme_train))
rsme_test = math.sqrt(mean_squared_error(y_test, pred_test[:,0]))
print('Test Score: %.2f RMSE' % (rsme_test))

In [ ]:
print(len(ts_train), len(pred_train), len(y_train))
len(ts_test), len(pred_test), len(y_test) 

In [ ]:
# shift train predictions for plotting
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features
pred_train_shifted = np.empty_like(ts_all)
print(pred_train_shifted.size)
pred_train_shifted[:, :] = np.nan
# train predictions start at position window_size + 1 (or window_size, if counting from 0)
pred_train_shifted[window_size : len(pred_train) + window_size, :] = pred_train
pred_train_shifted[:13]

In [ ]:
# shift test predictions for plotting
window_size = lstm_num_timesteps if use_LSTM else mlp_num_features
pred_test_shifted = np.empty_like(ts_all)
pred_test_shifted[:, :] = np.nan
pred_test_shifted[len(pred_train) + (window_size * 2) : len_overall + 1, :] = pred_test
pred_test_shifted[-13:]

In [ ]:
plt.plot(ts_all)
plt.plot(pred_train_shifted)
plt.plot(pred_test_shifted)
plt.show()

In [ ]:
plot_start = -100
plot_end = -1
plt.plot(ts_all[plot_start:plot_end])
plt.plot(pred_train_shifted[plot_start:plot_end])
plt.plot(pred_test_shifted[plot_start:plot_end])
plt.savefig(filename + '_lstm_' + str(use_LSTM) + '_stateful_' + str(lstm_stateful) + '_batchsize_' + str(batch_size) + '_window_' + str(window_size) +
            '_epochs_' + str(num_epochs) + '_2layers_' + str(lstm_stack_layers) + '.png')
plt.show()
